In [25]:
import sys
import json
sys.path.append('..')
sys.path.append('../questions_construction')

from src.common import *
from model_performances import *
from copy import deepcopy

import pandas as pd

EVERYTHING_POSTFIX = 'everything'

In [40]:
def to_latex(results_all, answer_type, plan_length,  domain = ALL_DOMAINS_KEY):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER : 'T/F',
        FREE_ANSWER : 'Free'
    }
    
    index = []
    data = []
    for question_category in QUESTION_CATEGORIES + [ALL_CATEGORIES_KEY]:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
            data_columns = {}
            for model_name in PROMPT_MODEL_NAMES:
                for prompt_type in PROMPT_TYPES:
                    instance = filter_results(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type)
                    if not len(instance) == 1:
                        raise ValueError(f'len(instance) == {len(instance)}')
                    data_columns[(model_name, prompt_type)] = instance[0][SK_RESULT]
            data.append(data_columns)
            
    df = pd.DataFrame(data, index = index)
    latex_table = df.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format)
    latex_table = latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')
    return latex_table

In [41]:
answer_type = TRUE_FALSE_ANSWER
results_all = open_jsonl(os.path.join(STATISTICS_PATH, f'{answer_type}.jsonl'))
latex_table_all = '\n'.join([to_latex(results_all, answer_type, plan_length) for plan_length in PLAN_LENGTHS])



/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_22343/948724066.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True,
/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_22343/948724066.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True,
/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_22343/948724066.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expect

In [42]:
print(latex_table_all)

\midrule
(1, object\_tracking, R, T/F)         &                      0.65 &                  0.69 &                      0.46 &                          0.62 &                      0.31 &                          0.38 \\
(1, object\_tracking, No R, T/F)      &                      0.54 &                  0.62 &                      0.75 &                          0.46 &                      0.29 &                          0.62 \\
(1, fluent\_tracking, R, T/F)         &                      0.54 &                  0.80 &                      0.80 &                          0.65 &                      0.30 &                          0.47 \\
(1, fluent\_tracking, No R, T/F)      &                      0.71 &                  0.66 &                      0.70 &                          0.63 &                      0.22 &                          0.44 \\
(1, state\_tracking, R, T/F)          &                      0.62 &                  0.80 &                      0.77 &                    